In [1]:
import requests
from bs4 import BeautifulSoup
import random
import smtplib
import os
from email.message import EmailMessage
import docx
import pandas as pd

In [2]:
state_cap = requests.get("https://en.wikipedia.org/wiki/List_of_capitals_of_states_of_Nigeria", 'lxml')
souped = BeautifulSoup(state_cap.text)

In [3]:
stat_nig = []
cap_nig = []

In [4]:
for x in range(0,72,2):
    stat_nig.append(souped.findAll('table', class_="wikitable")[0].findAll('tbody')[0].findAll('td')[x].findAll('a')[0].text)
    cap_nig.append(souped.findAll('table', class_="wikitable")[0].findAll('tbody')[0].findAll('td')[x+1].findAll('a')[0].text)

In [6]:
state_cap = dict(zip(stat_nig, cap_nig))

In [7]:
#state, capital = random.choice(list(state_cap.items()))

In [8]:
smtp = smtplib.SMTP_SSL('smtp.gmail.com', 465)
username = os.environ.get('User_name')
user_password = os.environ.get('User_password')

In [9]:
stud = pd.read_csv('Students_contact.csv')

In [10]:
def questPapers(studentName, email):
    doc = docx.Document()
    ansDoc = docx.Document()
    doc.add_heading(f'This question is for {studentName}', 2)
    states = random.sample(list(state_cap.keys()), 21)
    for i in range(1, 21):
        doc.add_paragraph(f'Question {i}:')
        doc.add_paragraph(f'What is the capital of {states[i]}')
        answer = state_cap[states[i]]
        opt = [answer]
        letter = ['A', 'B', 'C', 'D']
        for el in range(4):
            opt.append(random.choice(list(state_cap.values())))
            if len(opt) != len(set(opt)):
                opt = set(opt)
                opt = list(opt)
                opt.append(random.choice(list(state_cap.values())))
        opt.pop()
        random.shuffle(opt)
        t_opt = dict(zip(letter, opt))
        oopt = letter[opt.index(answer)]
        ansDoc.add_paragraph(f'The answer to question {i} is: {oopt}')
        for options, value in t_opt.items():
            doc.add_paragraph(f'{options}) {value} ')
            doc.add_paragraph('\n')    
            doc.save('testttttt.docx')
    ansDoc.save('answers.docx')
    msg = EmailMessage()
    msg['TO'] = email
    msg['From'] = username
    msg['Subject'] = 'This is your question'
    msg.set_content ('''Enjoy these questions''')
    with open('testttttt.docx', 'rb') as f:
        content = f.read()
        name = f.name
        msg.add_attachment(content, maintype='application', subtype ='msword', filename=name)
    with open('answers.docx', 'rb') as g:
        acontent = g.read()
        aname = g.name
        msg.add_attachment(acontent, maintype="application" ,subtype='msword', filename=aname)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(username, user_password)
        smtp.send_message(msg)

In [11]:
for row in stud.itertuples(index=True, name='Pandas'):
    questPapers(row.NAME, row.EMAIL)